<a href="https://colab.research.google.com/github/Daniel-hp/LSTM/blob/master/Resultados%20del%20entrenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Entrenamiento de la red sobre los nuevos videos**

In [4]:
from numpy.random import seed
seed(1996)
from tensorflow import set_random_seed
set_random_seed(1996)
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import  TimeseriesGenerator
from keras.layers import LSTM, Conv1D, MaxPool1D, Flatten, CuDNNGRU, CuDNNLSTM, Flatten
from hyperopt import Trials, STATUS_OK, tpe
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
import numpy as np
import keras as keras
from keras.optimizers import  rmsprop

X_train, Y_train, X_test, Y_test = np.load('/content/gdrive/My Drive/openpose_lstm/trainX.npy'), np.load('/content/gdrive/My Drive/openpose_lstm/y_train.npy'), np.load('/content/gdrive/My Drive/openpose_lstm/testX.npy'), np.load('/content/gdrive/My Drive/openpose_lstm/y_test.npy')

tam_input = 30

model = Sequential()
model.add(CuDNNLSTM(10, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(CuDNNLSTM(10, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Conv1D(filters=15, kernel_size=15 ))
model.add(MaxPool1D())
model.add(Flatten())
model.add(Dense(28, activation='softmax'))
opt = rmsprop(lr=0.001, rho=0.9)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=opt)
print(model.summary())
batch_size = 256
checkpoint = keras.callbacks.ModelCheckpoint('model_lstm2{epoch:08d}.h5', period=350)
history = model.fit(x=X_train, y= Y_train, epochs=150, batch_size=batch_size, validation_data=(X_test, Y_test) )

model.save('/content/gdrive/My Drive/openpose_lstm/modelo_def.h5')
validation_acc = np.amax(history.history['val_acc'])
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('/content/gdrive/My Drive/openpose_lstm/acc2.png')

plt.clf()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('/content/gdrive/My Drive/openpose_lstm/loss2.png')

print('Best validation acc of epoch:', validation_acc)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_3 (CuDNNLSTM)     (None, 2604, 10)          560       
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (None, 2604, 10)          880       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2590, 15)          2265      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1295, 15)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 19425)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 28)                543928    
Total params: 547,633
Trainable params: 547,633
Non-trainable params: 0
________________________________________________

### Gráficas de pérdida y de precisión del modelo, respectivamente

![perdida](https://drive.google.com/uc?id=1-G_JepAV2UH1MvP2NMHQPsQ_eH4TSUB7)
![precisión](https://drive.google.com/uc?id=1-Gxl6WIPG2P8KB0Lu3q5wKs_MHqKNFSX)



In [10]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


# generate 2d classification dataset
X, y = np.load('/content/gdrive/My Drive/openpose_lstm/devX.npy'), np.load('/content/gdrive/My Drive/openpose_lstm/y_dev.npy')

# define and fit the final model

model = load_model('/content/gdrive/My Drive/openpose_lstm/modelo_def.h5')

ynew = model.predict_classes(X)
# show the inputs and predicted outputs

print("Longitud conjunto = " + str(X.shape))
y_pred = []
y_true = []
class_names =  np.array(["Agua", "Alto", "Apagar", "Ayudar", "Campo", "Crecer", "Cuerpo", "Despedir", "Dinero", "Escribir", "Foto", "Fruto", "Hacer", "LavarCara", "Madera", "Miedo", "Morder", "Nariz", "Padrino","Película", "Persona", "Primero", "Quedar", "Saber", "Sangre", "Señorita", "Suerte", "Toro"])
tag_to_indx = {"Agua": 1, "Alto": 2, "Apagar": 3, "Ayudar": 4, "Campo": 5, "Crecer": 6, "Cuerpo": 7, "Despedir": 8,
               "Dinero": 9, "Escribir": 10, "Foto": 11, "Fruto": 12, "Hacer": 13, "LavarCara": 14, "Madera": 15,
               "Miedo": 16, "Morder": 17, "Nariz": 18, "Padrino": 19, "Película": 20, "Persona": 21, "Primero": 22,
               "Quedar": 23, "Saber": 24, "Sangre": 25, "Señorita": 26, "Suerte": 27, "Toro": 28}
indx_to_tag = {1: "Agua", 2: "Alto", 3: "Apagar", 4: "Ayudar", 5: "Campo", 6: "Crecer", 7: "Cuerpo", 8: "Despedir",
               9: "Dinero", 10: "Escribir", 11: "Foto", 12: "Fruto", 13: "Hacer", 14: "LavarCara", 15: "Madera",
               16: "Miedo", 17: "Morder", 18: "Nariz", 19: "Padrino", 20: "Película", 21: "Persona", 22: "Primero",
               23: "Quedar", 24: "Saber", 25: "Sangre", 26: "Señorita", 27: "Suerte", 28: "Toro"}
for i in range(len(X)):
    y_true.append(indx_to_tag[ynew[i] + 1])
    y_pred.append(indx_to_tag[np.argmax(y[i])+1])

cnf_matrix = confusion_matrix(y_true, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(18.0, 15.0))
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

plt.savefig('/content/gdrive/My Drive/openpose_lstm/matriz.png')
plt.clf()
# Plot normalized confusion matrix
plt.figure(figsize=(18.0, 15.0))
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.savefig('/content/gdrive/My Drive/openpose_lstm/matriz_norm.png')




Longitud conjunto = (1131, 2604, 2)
Confusion matrix, without normalization
[[32  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0 49  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0 50  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  1  0
   0  0  0  0]
 [ 0  0  0 44  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0 32  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  1  0  0  0 40  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  1  0]
 [ 0  0  0  0  0  0 35  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
   0  0  0  0]
 [ 0  0  0  0  0  0  0 33  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0  0  0  0  0 47  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 39  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 41

### Matriz de confusión sin normalizar y normalizada, respectivamente

![matriz](https://drive.google.com/uc?id=1oK-1piVnWd_tmZOLcG-hfCfe3L_d3Fdb)
![normalizada](https://drive.google.com/uc?id=1MsvHezBaScuAjwZI3mwR60NpkDJtiPLm)


### **Entrenamiento de la red sobre los videos anteriores**

In [0]:

from numpy.random import seed
seed(1996)
from tensorflow import set_random_seed
set_random_seed(1996)
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import  TimeseriesGenerator
from keras.layers import LSTM, Conv1D, MaxPool1D, Flatten, CuDNNGRU, CuDNNLSTM, Flatten
from hyperopt import Trials, STATUS_OK, tpe
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
import numpy as np
import keras as keras
from keras.optimizers import  rmsprop

X_train, Y_train, X_test, Y_test = np.load('/content/gdrive/My Drive/openpose_lstm/version_anterior/trainX.npy'), np.load('/content/gdrive/My Drive/openpose_lstm/version_anterior/y_train.npy'), np.load('/content/gdrive/My Drive/openpose_lstm/version_anterior/testX.npy'), np.load('/content/gdrive/My Drive/openpose_lstm/version_anterior/y_test.npy')

tam_input = 30
"""
model = Sequential()
model.add(LSTM(15, return_sequences=True, input_shape=(tam_input, X_train.shape[1])))
model.add(LSTM(15, input_shape=(tam_input, X_train.shape[1])))
model.add(Dense(28, activation='softmax'))

"""
model = Sequential()
model.add(CuDNNLSTM(10, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(CuDNNLSTM(10, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Conv1D(filters=15, kernel_size=15 ))
model.add(MaxPool1D())
model.add(Flatten())
model.add(Dense(28, activation='softmax'))
opt = rmsprop(lr=0.001, rho=0.9)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=opt)
print(model.summary())
batch_size = 256
checkpoint = keras.callbacks.ModelCheckpoint('model_lstm2{epoch:08d}.h5', period=350)
history = model.fit(x=X_train, y= Y_train, epochs=150, batch_size=batch_size, validation_data=(X_test, Y_test) )

model.save('/content/gdrive/My Drive/openpose_lstm/version_anterior/modelo_def.h5')
validation_acc = np.amax(history.history['val_acc'])
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('/content/gdrive/My Drive/openpose_lstm/version_anterior/acc2.png')

plt.clf()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('/content/gdrive/My Drive/openpose_lstm/version_anterior/loss2.png')

print('Best validation acc of epoch:', validation_acc)



Using TensorFlow backend.








Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 21948, 10)         560       
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 21948, 10)         880       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 21934, 15)         2265      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 10967, 15)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 164505)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 28)                4606168   
Total params: 4,609,873
Trainable params: 4,609,873
Non-trainable params: 0
______________________________________

### Gráficas de pérdida y de precisión del modelo, respectivamente

![alt text](https://drive.google.com/uc?id=1ge0VWY8u09DpwboUz9bXUHROG6xUk2um)
![alt text](https://drive.google.com/uc?id=1TJlJ0DC05g5zLXNCNkTOP4B2VV6pSKYQ)

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


# generate 2d classification dataset
X, y = np.load('/content/gdrive/My Drive/openpose_lstm/version_anterior/devX.npy'), np.load('/content/gdrive/My Drive/openpose_lstm/version_anterior/y_dev.npy')

# define and fit the final model

model = load_model('/content/gdrive/My Drive/openpose_lstm/version_anterior/modelo_def.h5')

ynew = model.predict_classes(X)
# show the inputs and predicted outputs

print("Longitud conjunto = " + str(X.shape))
y_pred = []
y_true = []
class_names =  np.array(["Agua", "Alto", "Apagar", "Ayudar", "Campo", "Crecer", "Cuerpo", "Despedir", "Dinero", "Escribir", "Foto", "Fruto", "Hacer", "LavarCara", "Madera", "Miedo", "Morder", "Nariz", "Padrino","Película", "Persona", "Primero", "Quedar", "Saber", "Sangre", "Señorita", "Suerte", "Toro"])
tag_to_indx = {"Agua": 1, "Alto": 2, "Apagar": 3, "Ayudar": 4, "Campo": 5, "Crecer": 6, "Cuerpo": 7, "Despedir": 8,
               "Dinero": 9, "Escribir": 10, "Foto": 11, "Fruto": 12, "Hacer": 13, "LavarCara": 14, "Madera": 15,
               "Miedo": 16, "Morder": 17, "Nariz": 18, "Padrino": 19, "Película": 20, "Persona": 21, "Primero": 22,
               "Quedar": 23, "Saber": 24, "Sangre": 25, "Señorita": 26, "Suerte": 27, "Toro": 28}
indx_to_tag = {1: "Agua", 2: "Alto", 3: "Apagar", 4: "Ayudar", 5: "Campo", 6: "Crecer", 7: "Cuerpo", 8: "Despedir",
               9: "Dinero", 10: "Escribir", 11: "Foto", 12: "Fruto", 13: "Hacer", 14: "LavarCara", 15: "Madera",
               16: "Miedo", 17: "Morder", 18: "Nariz", 19: "Padrino", 20: "Película", 21: "Persona", 22: "Primero",
               23: "Quedar", 24: "Saber", 25: "Sangre", 26: "Señorita", 27: "Suerte", 28: "Toro"}
for i in range(len(X)):
    y_true.append(indx_to_tag[ynew[i] + 1])
    y_pred.append(indx_to_tag[np.argmax(y[i])+1])

cnf_matrix = confusion_matrix(y_true, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(18.0, 15.0))
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

plt.savefig('/content/gdrive/My Drive/openpose_lstm/version_anterior/matriz.png')
plt.clf()
# Plot normalized confusion matrix
plt.figure(figsize=(18.0, 15.0))
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.savefig('/content/gdrive/My Drive/openpose_lstm/version_anterior/matriz_norm.png')




Longitud conjunto = (1120, 21948, 2)
Confusion matrix, without normalization
[[42  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0 41  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0 33  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0 42  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0 32  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0  0 37  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0  0  0 45  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0  0  0  0 44  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0  0  0  0  0 36  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 36  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 4

### Matriz de confusión sin normalizar y normalizada, respectivamente

![matriz](https://drive.google.com/uc?id=1-3AykeOW4t54hak_LT9-6Bvj3h2Dt8xt)
![normalizada](https://drive.google.com/uc?id=1-6b8dw5cumKguyhmrXykStatWwH1Zo7W)